# HW 1  - 01_intro

See [HW1 assignment in the course repo](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md)


## Notes

Setup week for elsticsearch and OpenAI. Everything's working.

I extracted functions into `lib/my_elasticsearch.py` and `lib/my_llm.py` .
They look like something that will be used in upcoming weeks, future me will appreciate the head start.

Week 2 introduces other LLM services, which will clarify what a nice generic LLM 
client interface looks like. 

Didn't incorporate  minsearch.py into `my_elasticsearch.py`, I doubt we'll come back to it. 

In [2]:
import os

DIR = os.get_cwd()
from lib import my_elasticsearch

# One-time setup to add documents to elasticsearch

In [5]:
import json

output = 'scrubbed_docs.json'
# v1 - downloaded the search_workshop repo that this came from 
# with open('../search_workshop/documents.json', 'rt') as fp:
#     raw_documents = json.load(fp)

# for item in raw_documents:
#     c = item['course']
#     for doc in item['documents']:
#         doc['course'] = c

# save for next time
# with open(f"{DIR}/{output}", 'w') as op:
#     json.dump(raw_documents, op)

# going forward: load from file
with open(output, 'rt') as fp:
    raw_documents = json.load(fp)

# Flatten them to one list
documents = raw_documents
all_docs = []
for category in documents:
    print(f"Category: {category['course']}")
    all_docs.extend(category['documents'])
    print(len(all_docs))

In [23]:
# TODO save the full flat list so I don't have to reprocess? knock on wood

# Import to elasticsearch

In [3]:
es_client = my_elasticsearch.get_client('http://localhost:9200')
es_client.info()  # anybody home?

ObjectApiResponse({'name': 'eb170f70b4fa', 'cluster_name': 'docker-cluster', 'cluster_uuid': '6ZPf_FQxSomgHxU1wh4P7w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
# create the index
index_name = 'course_questions'
my_elasticsearch.create_index(index_name)
my_elasticsearch.load_documents(all_docs, index_name)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course_questions/vd-aOgqZTYGrLJFD6iRpMQ] already exists')

# Q3

In [9]:
query = "How do I execute a command in a running docker container?"
fields = ["question^4", "text"]
answers = my_elasticsearch.search(query, fields)
sorted([item['_score'] for item in answers])

In [96]:
answers = my_elasticsearch.search(query, fields, course="machine-learning-zoomcamp", results=3)

# Q4

In [12]:
q4_answers = my_elasticsearch.search(query, fields, course="machine-learning-zoomcamp", results=3)
# q4_answers[-1]

{'_index': 'course_questions',
 '_id': 'uMRLapABhvmccvXLswTb',
 '_score': 49.938507,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

# Q5

In [16]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

question = 'How do I execute a command in a running docker container?'

In [17]:
context = []

context_template = """
Q: {question}
A: {text}
""".strip()

for q in q4_answers:
    context.append(context_template.format(question=q['_source']['question'], text=q['_source']['text']))
context = "\n\n".join(context)

prompt = prompt_template.format(question=question, context=context)

In [18]:
# Q5: how long is the prompt?
len(prompt)

1462

In [19]:
# How many words in the prompt?
len(prompt.split())
# 236


236

In [20]:
# Q6  How many tokens?

In [21]:
from lib import my_llm

my_llm.count_tokens(prompt)

322